In [ ]:
!wget 'http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2'
!bzip2 -dk '/content/shape_predictor_68_face_landmarks.dat.bz2' 

--2020-08-22 03:41:10--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  17.3MB/s    in 3.5s    

2020-08-22 03:41:14 (17.3 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]



In [ ]:
!pip install autocrop

     |████████████████████████████████| 163kB 3.4MB/s 
     |████████████████████████████████| 49.0MB 83kB/s 
  Created wheel for autocrop: filename=autocrop-1.0.2-cp36-none-any.whl size=157696 sha256=e7f086847018d83fa5c6b4b554d3c42d2039a85d01de072c4fc2e867fa10c51c
  Stored in directory: /root/.cache/pip/wheels/8e/5a/8b/e173ae548ba963c0bd47500b8631196acc878e4666ca121ebc
Successfully built autocrop
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30


In [ ]:
!pip install tensorflow==1.13.0rc1
!pip install scipy==1.1.0

     |████████████████████████████████| 31.2MB 131kB/s 
ERROR: umap-learn 0.4.6 has requirement scipy>=1.3.1, but you'll have scipy 1.1.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import dlib
import math
import urllib3
import glob
import os
import shutil

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from scipy import misc
import sys
import argparse
import tensorflow as tf
import src.facenet as facenet
import src.align as align
import src.align.detect_face
import random
from time import sleep

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [ ]:
PREDICTOR_PATH = '/content/shape_predictor_68_face_landmarks.dat'

In [ ]:
face_detector = dlib.get_frontal_face_detector()
face_landmark_predictor = dlib.shape_predictor(PREDICTOR_PATH)

In [ ]:
def get_faces(img, face_detector, face_landmark_predictor):
  face_rects = face_detector(img, 2)
  faces = []
  
  face_img = img.copy()
  H, W, _ = face_img.shape

  if len(face_rects) < 1:
    return faces

  for i, face_rect in enumerate(face_rects):
    left, top, right, bottom = face_rect.left(), face_rect.top(), face_rect.right(), face_rect.bottom()

    rect = dlib.rectangle(left, top, right, bottom)
    landmarks = face_landmark_predictor(img, rect)

    top_left = (left, top)
    bottom_right = (right, bottom)
    
    face_img = cv2.rectangle(face_img, top_left, bottom_right, color=(255, 255, 0), thickness=2) 
    
    border = int(2 * min(abs(top - bottom), abs(left - right)))
    # border = 0
    top_crop = top - border if top - border > 0 else 0
    bottom_crop = bottom + border if bottom + border < H else H 
    left_crop = left - border if left - border > 0 else 0
    right_crop = right + border if right + border < W else W

    img_rect = face_img[top_crop:bottom_crop, left_crop:right_crop]

    landmarks_offsetted = []
    landmarks_actual = []
    for i, landmark in enumerate(landmarks.parts()):
      x, y, = landmark.x, landmark.y
      landmarks_actual.append((x, y))
      landmarks_offsetted.append((x, y))
      img_rect = cv2.circle(img_rect, (x, y), radius=1, color=(255, 0, 0), thickness=2)

    faces.append({'idx': i, 
                  'rect_xy': (left, top, right, bottom), 
                  'rect_xy_offsetted': (left_crop, top_crop, right_crop, bottom_crop), 
                  'landmarks_offsetted': landmarks_offsetted,
                  'landmarks': landmarks_actual,
                  'img': img,
                  'rect': img[top_crop:bottom_crop, left_crop:right_crop]})
    
    return faces

In [ ]:
glob.glob("/content/drive/My Drive/face_rec_data/*")

['/content/drive/My Drive/face_rec_data/Dinesh_K.mp4',
 '/content/drive/My Drive/face_rec_data/Siva_Sankar.mp4',
 '/content/drive/My Drive/face_rec_data/Sahal_K.mp4',
 '/content/drive/My Drive/face_rec_data/Amer_Miracle.mp4',
 '/content/drive/My Drive/face_rec_data/Siva_Sankar160',
 '/content/drive/My Drive/face_rec_data/Sahal_K']

In [ ]:
media_file = "/content/drive/My Drive/face_rec_data/Siva_Sankar.mp4"

dir, _ = os.path.splitext(media_file)
if os.path.exists(dir):
  shutil.rmtree(dir)
os.mkdir(dir)

cap = cv2.VideoCapture(media_file)
count = 0

# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")

# Read until video is completed
while(cap.isOpened() and count < 200):
  # Capture frame-by-frame
  ret, img = cap.read()
  if ret == True:
    img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
    # plt.imshow(img)
    faces = get_faces(img, face_detector, face_landmark_predictor)
    # print(img.shape)
    if len(faces) >= 0:
      img_path = os.path.join(dir, f'img_{count:03d}.jpg')
      cv2.imwrite(img_path, img)
      print(f'Writing {img_path} ...')
      count += 1
  # Break the loop
  else: 
    break

# When everything done, release the video capture object
cap.release()

Writing /content/drive/My Drive/face_rec_data/Siva_Sankar/img_000.jpg ...
Writing /content/drive/My Drive/face_rec_data/Siva_Sankar/img_001.jpg ...
Writing /content/drive/My Drive/face_rec_data/Siva_Sankar/img_002.jpg ...
Writing /content/drive/My Drive/face_rec_data/Siva_Sankar/img_003.jpg ...
Writing /content/drive/My Drive/face_rec_data/Siva_Sankar/img_004.jpg ...
Writing /content/drive/My Drive/face_rec_data/Siva_Sankar/img_005.jpg ...
Writing /content/drive/My Drive/face_rec_data/Siva_Sankar/img_006.jpg ...
Writing /content/drive/My Drive/face_rec_data/Siva_Sankar/img_007.jpg ...
Writing /content/drive/My Drive/face_rec_data/Siva_Sankar/img_008.jpg ...
Writing /content/drive/My Drive/face_rec_data/Siva_Sankar/img_009.jpg ...
Writing /content/drive/My Drive/face_rec_data/Siva_Sankar/img_010.jpg ...
Writing /content/drive/My Drive/face_rec_data/Siva_Sankar/img_011.jpg ...
Writing /content/drive/My Drive/face_rec_data/Siva_Sankar/img_012.jpg ...
Writing /content/drive/My Drive/face_r

In [ ]:
!autocrop -i "/content/drive/My Drive/face_rec_data/Siva_Sankar" -o "/content/drive/My Drive/face_rec_data/autocrop" -r "/content/drive/My Drive/face_rec_data/reject" -w 720 -H 720 --facePercent 80

Processing images in folder: /content/drive/My Drive/face_rec_data/Siva_Sankar
Face detected:    /content/drive/My Drive/face_rec_data/autocrop/img_000.jpg
Face detected:    /content/drive/My Drive/face_rec_data/autocrop/img_001.jpg
Face detected:    /content/drive/My Drive/face_rec_data/autocrop/img_002.jpg
Face detected:    /content/drive/My Drive/face_rec_data/autocrop/img_003.jpg
Face detected:    /content/drive/My Drive/face_rec_data/autocrop/img_004.jpg
Face detected:    /content/drive/My Drive/face_rec_data/autocrop/img_005.jpg
Face detected:    /content/drive/My Drive/face_rec_data/autocrop/img_006.jpg
Face detected:    /content/drive/My Drive/face_rec_data/autocrop/img_007.jpg
Face detected:    /content/drive/My Drive/face_rec_data/autocrop/img_008.jpg
Face detected:    /content/drive/My Drive/face_rec_data/autocrop/img_009.jpg
Face detected:    /content/drive/My Drive/face_rec_data/autocrop/img_010.jpg
Face detected:    /content/drive/My Drive/face_rec_data/autocrop/img_011.j

In [ ]:
!rm -rf "/content/drive/My Drive/face_data_train/Siva_Sankar"
#!rm -rf "/content/drive/My Drive/face_rec_data/Siva_Sankar160/Amar_Miracle"

In [ ]:
!git clone https://github.com/davidsandberg/facenet.git

Cloning into 'facenet'...
remote: Enumerating objects: 3149, done.
remote: Total 3149 (delta 0), reused 0 (delta 0), pack-reused 3149
Receiving objects: 100% (3149/3149), 2.94 MiB | 13.86 MiB/s, done.
Resolving deltas: 100% (2230/2230), done.


In [ ]:
def align_for_training(args):

    sleep(random.random())
    output_dir = os.path.expanduser(args.output_dir)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    # Store some git revision info in a text file in the log directory
    # src_path,_ = os.path.split(os.path.realpath(__file__))
    # facenet.store_revision_info(src_path, output_dir, ' '.join(sys.argv))
    dataset = facenet.get_dataset(args.input_dir)
    
    print('Creating networks and loading parameters')
    
    with tf.Graph().as_default():
        gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=args.gpu_memory_fraction)
        sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
        with sess.as_default():
            # save np.load
            np_load_old = np.load
            # modify the default parameters of np.load
            np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

            pnet, rnet, onet = align.detect_face.create_mtcnn(sess, None)

            # restore np.load for future normal usage
            np.load = np_load_old
    
    minsize = 20 # minimum size of face
    threshold = [ 0.6, 0.7, 0.7 ]  # three steps's threshold
    factor = 0.709 # scale factor

    # Add a random key to the filename to allow alignment using multiple processes
    random_key = np.random.randint(0, high=99999)
    bounding_boxes_filename = os.path.join(output_dir, 'bounding_boxes_%05d.txt' % random_key)
    
    with open(bounding_boxes_filename, "w") as text_file:
        nrof_images_total = 0
        nrof_successfully_aligned = 0
        if args.random_order:
            random.shuffle(dataset)
        for cls in dataset:
            output_class_dir = os.path.join(output_dir, cls.name)
            if not os.path.exists(output_class_dir):
                os.makedirs(output_class_dir)
                if args.random_order:
                    random.shuffle(cls.image_paths)
            for image_path in cls.image_paths:
                nrof_images_total += 1
                filename = os.path.splitext(os.path.split(image_path)[1])[0]
                output_filename = os.path.join(output_class_dir, filename+'.png')
                print(image_path)
                if not os.path.exists(output_filename):
                    try:
                        img = misc.imread(image_path)
                    except (IOError, ValueError, IndexError) as e:
                        errorMessage = '{}: {}'.format(image_path, e)
                        print(errorMessage)
                    else:
                        if img.ndim<2:
                            print('Unable to align "%s"' % image_path)
                            text_file.write('%s\n' % (output_filename))
                            continue
                        if img.ndim == 2:
                            img = facenet.to_rgb(img)
                        img = img[:,:,0:3]
    
                        bounding_boxes, _ = align.detect_face.detect_face(img, minsize, pnet, rnet, onet, threshold, factor)
                        nrof_faces = bounding_boxes.shape[0]
                        if nrof_faces>0:
                            det = bounding_boxes[:,0:4]
                            det_arr = []
                            img_size = np.asarray(img.shape)[0:2]
                            if nrof_faces>1:
                                if args.detect_multiple_faces:
                                    for i in range(nrof_faces):
                                        det_arr.append(np.squeeze(det[i]))
                                else:
                                    bounding_box_size = (det[:,2]-det[:,0])*(det[:,3]-det[:,1])
                                    img_center = img_size / 2
                                    offsets = np.vstack([ (det[:,0]+det[:,2])/2-img_center[1], (det[:,1]+det[:,3])/2-img_center[0] ])
                                    offset_dist_squared = np.sum(np.power(offsets,2.0),0)
                                    index = np.argmax(bounding_box_size-offset_dist_squared*2.0) # some extra weight on the centering
                                    det_arr.append(det[index,:])
                            else:
                                det_arr.append(np.squeeze(det))

                            for i, det in enumerate(det_arr):
                                det = np.squeeze(det)
                                bb = np.zeros(4, dtype=np.int32)
                                bb[0] = np.maximum(det[0]-args.margin/2, 0)
                                bb[1] = np.maximum(det[1]-args.margin/2, 0)
                                bb[2] = np.minimum(det[2]+args.margin/2, img_size[1])
                                bb[3] = np.minimum(det[3]+args.margin/2, img_size[0])
                                cropped = img[bb[1]:bb[3],bb[0]:bb[2],:]
                                scaled = misc.imresize(cropped, (args.image_size, args.image_size), interp='bilinear')
                                nrof_successfully_aligned += 1
                                filename_base, file_extension = os.path.splitext(output_filename)
                                if args.detect_multiple_faces:
                                    output_filename_n = "{}_{}{}".format(filename_base, i, file_extension)
                                else:
                                    output_filename_n = "{}{}".format(filename_base, file_extension)
                                misc.imsave(output_filename_n, scaled)
                                text_file.write('%s %d %d %d %d\n' % (output_filename_n, bb[0], bb[1], bb[2], bb[3]))
                        else:
                            print('Unable to align "%s"' % image_path)
                            text_file.write('%s\n' % (output_filename))
                            
    print('Total number of images: %d' % nrof_images_total)
    print('Number of successfully aligned images: %d' % nrof_successfully_aligned)
            

def parse_arguments(argv):
    parser = argparse.ArgumentParser()
    
    parser.add_argument('input_dir', type=str, help='Directory with unaligned images.')
    parser.add_argument('output_dir', type=str, help='Directory with aligned face thumbnails.')
    parser.add_argument('--image_size', type=int,
        help='Image size (height, width) in pixels.', default=182)
    parser.add_argument('--margin', type=int,
        help='Margin for the crop around the bounding box (height, width) in pixels.', default=44)
    parser.add_argument('--random_order', 
        help='Shuffles the order of images to enable alignment using multiple processes.', action='store_true')
    parser.add_argument('--gpu_memory_fraction', type=float,
        help='Upper bound on the amount of GPU memory that will be used by the process.', default=1.0)
    parser.add_argument('--detect_multiple_faces', type=bool,
                        help='Detect and align multiple faces per image.', default=False)
    return parser.parse_args(argv)


args = argparse.Namespace(input_dir="/content/drive/My Drive/face_data_train", 
                          output_dir= "/content/drive/My Drive/face_rec_data/Siva_Sankar160",
                          image_size=160,
                          margin=32,
                          random_order=True,
                          gpu_memory_fraction=0.25,
                          detect_multiple_faces=False)
align_for_training(args)

Creating networks and loading parameters
/content/drive/My Drive/face_data_train/Sahal_K/img_000.jpg
/content/drive/My Drive/face_data_train/Sahal_K/img_001.jpg
/content/drive/My Drive/face_data_train/Sahal_K/img_002.jpg
/content/drive/My Drive/face_data_train/Sahal_K/img_003.jpg
/content/drive/My Drive/face_data_train/Sahal_K/img_004.jpg
/content/drive/My Drive/face_data_train/Sahal_K/img_005.jpg
/content/drive/My Drive/face_data_train/Sahal_K/img_006.jpg
/content/drive/My Drive/face_data_train/Sahal_K/img_007.jpg
/content/drive/My Drive/face_data_train/Sahal_K/img_008.jpg
/content/drive/My Drive/face_data_train/Sahal_K/img_009.jpg
/content/drive/My Drive/face_data_train/Sahal_K/img_010.jpg
/content/drive/My Drive/face_data_train/Sahal_K/img_011.jpg
/content/drive/My Drive/face_data_train/Sahal_K/img_012.jpg
/content/drive/My Drive/face_data_train/Sahal_K/img_013.jpg
/content/drive/My Drive/face_data_train/Sahal_K/img_014.jpg
/content/drive/My Drive/face_data_train/Sahal_K/img_015.jpg

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:54: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


Unable to align "/content/drive/My Drive/face_data_train/Sahal_K/img_114.jpg"
/content/drive/My Drive/face_data_train/Sahal_K/img_116.jpg
/content/drive/My Drive/face_data_train/Sahal_K/img_117.jpg
/content/drive/My Drive/face_data_train/Sahal_K/img_118.jpg
/content/drive/My Drive/face_data_train/Sahal_K/img_119.jpg
/content/drive/My Drive/face_data_train/Sahal_K/img_120.jpg
/content/drive/My Drive/face_data_train/Sahal_K/img_121.jpg
/content/drive/My Drive/face_data_train/Sahal_K/img_122.jpg
/content/drive/My Drive/face_data_train/Sahal_K/img_123.jpg
/content/drive/My Drive/face_data_train/Sahal_K/img_124.jpg
/content/drive/My Drive/face_data_train/Sahal_K/img_125.jpg
/content/drive/My Drive/face_data_train/Sahal_K/img_126.jpg
/content/drive/My Drive/face_data_train/Sahal_K/img_127.jpg
/content/drive/My Drive/face_data_train/Sahal_K/img_128.jpg
/content/drive/My Drive/face_data_train/Sahal_K/img_129.jpg
/content/drive/My Drive/face_data_train/Sahal_K/img_130.jpg
/content/drive/My Driv

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:95: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:102: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


/content/drive/My Drive/face_data_train/Siva_Sankar/img_021.jpg
/content/drive/My Drive/face_data_train/Siva_Sankar/img_022.jpg
/content/drive/My Drive/face_data_train/Siva_Sankar/img_023.jpg
/content/drive/My Drive/face_data_train/Siva_Sankar/img_024.jpg
/content/drive/My Drive/face_data_train/Siva_Sankar/img_025.jpg
/content/drive/My Drive/face_data_train/Siva_Sankar/img_026.jpg
/content/drive/My Drive/face_data_train/Siva_Sankar/img_027.jpg
/content/drive/My Drive/face_data_train/Siva_Sankar/img_028.jpg
/content/drive/My Drive/face_data_train/Siva_Sankar/img_029.jpg
/content/drive/My Drive/face_data_train/Siva_Sankar/img_030.jpg
/content/drive/My Drive/face_data_train/Siva_Sankar/img_031.jpg
/content/drive/My Drive/face_data_train/Siva_Sankar/img_032.jpg
/content/drive/My Drive/face_data_train/Siva_Sankar/img_033.jpg
/content/drive/My Drive/face_data_train/Siva_Sankar/img_034.jpg
/content/drive/My Drive/face_data_train/Siva_Sankar/img_035.jpg
/content/drive/My Drive/face_data_train/

In [ ]:
!zip -r "data.zip" "/content/drive/My Drive/face_rec_data/Siva_Sankar160"

  adding: content/drive/My Drive/face_rec_data/Siva_Sankar160/ (stored 0%)
  adding: content/drive/My Drive/face_rec_data/Siva_Sankar160/bounding_boxes_26582.txt (stored 0%)
  adding: content/drive/My Drive/face_rec_data/Siva_Sankar160/bounding_boxes_30990.txt (stored 0%)
  adding: content/drive/My Drive/face_rec_data/Siva_Sankar160/bounding_boxes_78295.txt (stored 0%)
  adding: content/drive/My Drive/face_rec_data/Siva_Sankar160/bounding_boxes_00031.txt (stored 0%)
  adding: content/drive/My Drive/face_rec_data/Siva_Sankar160/Siva_Sankar/ (stored 0%)
  adding: content/drive/My Drive/face_rec_data/Siva_Sankar160/Siva_Sankar/img_000.png (deflated 0%)
  adding: content/drive/My Drive/face_rec_data/Siva_Sankar160/Siva_Sankar/img_001.png (deflated 0%)
  adding: content/drive/My Drive/face_rec_data/Siva_Sankar160/Siva_Sankar/img_002.png (deflated 0%)
  adding: content/drive/My Drive/face_rec_data/Siva_Sankar160/Siva_Sankar/img_003.png (deflated 0%)
  adding: content/drive/My Drive/face_rec_